<a href="https://colab.research.google.com/github/ShimitaRudra/DataAnalytics-Thesis/blob/main/Drug_reviews_DA_project_BCB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers >> NULL

### Already installed packages

In [ ]:
#!pip install -U spacy

In [ ]:
#!pip install -U spacy[cuda118]

In [ ]:
#!pip install scispacy

In [ ]:
#!pip install -U scispacy

In [ ]:
#!pip install pyarrow

In [ ]:
#!pip install fastparquet

In [ ]:
#!pip install spacy-transformers >> a

In [ ]:
#pip install '/content/drive/MyDrive/Colab Notebooks/Data Analytics Project/en_ner_bc5cdr_md-0.5.1.tar.gz'

### Import statementss

In [ ]:
import pandas as pd
import numpy as np
#import spacy
#import spacy_transformers
#import scispacy
from spacy import displacy
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, classification_report
from transformers import AutoTokenizer, AutoModel, AdamW, get_linear_schedule_with_warmup, AutoConfig

import torch
import torch.nn as nn
import pickle

from tqdm.notebook import tqdm
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

import matplotlib.pyplot as plt

In [ ]:
print("CUDA Version:", torch.version.cuda)

CUDA Version: 11.8


### Save and Load Balanced + Sciscpacy + Tokenize Filtered df

In [ ]:
#Load df
df=pd.read_parquet('/content/drive/MyDrive/Colab Notebooks/Data Analytics Project/df_clean_save/df_bal_scispacy_filter.parquet')

In [ ]:
df.info() #118699 | memory usage: 7.2+ MB | 7 columns

<class 'pandas.core.frame.DataFrame'>
Int64Index: 118699 entries, 0 to 118703
Data columns (total 7 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   drugName         118699 non-null  object
 1   condition        118699 non-null  object
 2   rating           118699 non-null  int64 
 3   label            118699 non-null  int64 
 4   label_name       118699 non-null  object
 5   scispacy_review  118699 non-null  object
 6   bert_token_len   118699 non-null  int64 
dtypes: int64(3), object(4)
memory usage: 7.2+ MB


In [ ]:
df = df.rename(columns = {'scispacy_review':'review'})

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 118699 entries, 0 to 118703
Data columns (total 7 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   drugName        118699 non-null  object
 1   condition       118699 non-null  object
 2   rating          118699 non-null  int64 
 3   label           118699 non-null  int64 
 4   label_name      118699 non-null  object
 5   review          118699 non-null  object
 6   bert_token_len  118699 non-null  int64 
dtypes: int64(3), object(4)
memory usage: 7.2+ MB


### Extra Config

In [ ]:
#Following C. Col´on-Ruiz and I. Segura-Bedmar, “Comparing deep learning architectures for sentiment analysis on drug reviews,” Journal of
#Biomedical Informatics, vol. 110, p. 103539, 2020. [Online]. Available:
#https://www.sciencedirect.com/science/article/pii/S1532046420301672

class Config():
    epochs = 5
    batch_size = 200
    max_length = 250 #512
    lr = 0.001
    hidden_size = 128
    epsilon = 1e-8
    random_state=100
    add_special_tokens=True
    return_attention_mask=True
    padding='max_length'
    truncation=True
    return_token_type_ids=False
    return_tensors='pt'
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

config = Config()


In [ ]:
#Following B. Biseda and K. Mo, “Enhancing pharmacovigilance with drug reviews and
#social media,” arXiv preprint arXiv:2004.08731, 2020. 6, 8, 15
'''
class Config():
    epochs = 10
    batch_size = 32
    max_length = 128 #250
    lr = 2e-5
    hidden_size = 128
    epsilon = 1e-8
    random_state=100
    add_special_tokens=True
    return_attention_mask=True
    padding='max_length'
    truncation=True
    return_token_type_ids=False
    return_tensors='pt'
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

config = Config()
'''

### Config

In [ ]:
#Following A. Shiju and Z. He, “Classifying drug ratings using user reviews with
#transformer-based language models,” in 2022 IEEE 10th International Conference
#on Healthcare Informatics (ICHI), 2022, pp. 163–169. 7, 8, 15

class Config():
    epochs = 10
    batch_size = 6
    max_length = 512
    lr = 5e-5
    hidden_size = 128
    epsilon = 1e-8
    random_state=100
    add_special_tokens=True
    return_attention_mask=True
    padding='max_length'
    truncation=True
    return_token_type_ids=False
    return_tensors='pt'
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

config = Config()


In [ ]:
params = {
    "epochs":config.epochs,
    "batch_size":config.batch_size,
    "max_length":config.max_length,
    "lr":config.lr,
    "hidden_size":config.hidden_size,
    "eps":config.epsilon,
    "random_state":config.random_state,
    "add_special_tokens":config.add_special_tokens,
    "return_attention_mask":config.return_attention_mask,
    "padding":config.padding,
    "truncation":config.truncation,
    "return_token_type_ids":config.return_token_type_ids,
    "return_tensors":config.return_tensors,
    "device":str(config.device),
         }

In [ ]:
device = config.device
device

device(type='cuda', index=0)

In [ ]:
torch.cuda.is_available()

True

### Train Test split

In [ ]:
train_df, temp_df = train_test_split(df,test_size=0.3,random_state=100)

val_df, test_df = train_test_split(temp_df,test_size=0.5,random_state=100)
#70:15:15

In [ ]:
print(len(train_df['label'].unique()))
print("TrainDF:",train_df.shape)
print(len(val_df['label'].unique()))
print("Valdf:",val_df.shape)
print(len(test_df['label'].unique()))
print("Testdf:",test_df.shape)

3
TrainDF: (83089, 7)
3
Valdf: (17805, 7)
3
Testdf: (17805, 7)


In [ ]:
train_df.info() #83089 | memory usage: 5.1+ MB | 7 columns

<class 'pandas.core.frame.DataFrame'>
Int64Index: 83089 entries, 32168 to 38409
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   drugName        83089 non-null  object
 1   condition       83089 non-null  object
 2   rating          83089 non-null  int64 
 3   label           83089 non-null  int64 
 4   label_name      83089 non-null  object
 5   review          83089 non-null  object
 6   bert_token_len  83089 non-null  int64 
dtypes: int64(3), object(4)
memory usage: 5.1+ MB


### Bio Clinical BERT

In [ ]:
tokenizer_bcb = AutoTokenizer.from_pretrained('emilyalsentzer/Bio_ClinicalBERT')

model_bcb = AutoModel.from_pretrained('emilyalsentzer/Bio_ClinicalBERT')
#model = model.to(device)

In [ ]:
model_bcb

#### Batch Encode into input id, attention mask

In [ ]:
train_encoded = tokenizer_bcb.batch_encode_plus(train_df.review.values.tolist(),
                                                 add_special_tokens=config.add_special_tokens,
                                                 return_attention_mask=config.return_attention_mask,
                                                 padding=config.padding,
                                                 max_length=config.max_length,
                                                 truncation = config.truncation,
                                                 return_token_type_ids = config.return_token_type_ids,
                                                 return_tensors=config.return_tensors)

In [ ]:
val_encoded = tokenizer_bcb.batch_encode_plus(val_df.review.values.tolist(),
                                               add_special_tokens=config.add_special_tokens,
                                               return_attention_mask=config.return_attention_mask,
                                               padding=config.padding,
                                               max_length=config.max_length,
                                               truncation = config.truncation,
                                               return_token_type_ids = config.return_token_type_ids,
                                               return_tensors=config.return_tensors)

In [ ]:
test_encoded = tokenizer_bcb.batch_encode_plus(test_df.review.values.tolist(),
                                               add_special_tokens=config.add_special_tokens,
                                               return_attention_mask=config.return_attention_mask,
                                               padding=config.padding,
                                               max_length=config.max_length,
                                               truncation = config.truncation,
                                               return_token_type_ids = config.return_token_type_ids,
                                               return_tensors=config.return_tensors)

#### Integer input & mask to Tensor

In [ ]:
train_input = torch.tensor(train_encoded['input_ids'])
train_att_mask = torch.tensor(train_encoded['attention_mask'])
train_labels = torch.tensor(train_df.label.values)

val_input = torch.tensor(val_encoded['input_ids'])
val_att_mask = torch.tensor(val_encoded['attention_mask'])
val_labels = torch.tensor(val_df.label.values)

test_input = torch.tensor(test_encoded['input_ids'])
test_att_mask = torch.tensor(test_encoded['attention_mask'])
test_labels = torch.tensor(test_df.label.values)

<ipython-input-26-5b51f894ed33>:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_input = torch.tensor(train_encoded['input_ids'])
<ipython-input-26-5b51f894ed33>:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_att_mask = torch.tensor(train_encoded['attention_mask'])
<ipython-input-26-5b51f894ed33>:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  val_input = torch.tensor(val_encoded['input_ids'])
<ipython-input-26-5b51f894ed33>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detac

#### Creating Tensor data & DataLoaders

In [ ]:
train_dataset = TensorDataset(train_input, train_att_mask, train_labels)
val_dataset = TensorDataset(val_input, val_att_mask, val_labels)
test_dataset = TensorDataset(test_input, test_att_mask, test_labels)

train_dataloader = DataLoader(train_dataset, batch_size=config.batch_size,sampler=RandomSampler(train_dataset))
val_dataloader = DataLoader(val_dataset, batch_size=config.batch_size, sampler=SequentialSampler(val_dataset))
test_dataloader = DataLoader(test_dataset, batch_size=config.batch_size, sampler=SequentialSampler(test_dataset))

### Custom Model

In [ ]:
#Just to check
model_bcb_check = "emilyalsentzer/Bio_ClinicalBERT"
config2 = AutoConfig.from_pretrained(model_bcb_check)

hidden_size = config2.hidden_size
hidden_size #768

768

In [ ]:
class BERTBiGRUClassifier(nn.Module):
  def __init__(self, bert_model, tokenizer, hidden_size, num_classes, freeze_bert=True):
    super(BERTBiGRUClassifier, self).__init__()
    self.bert = bert_model
    self.tokenizer = tokenizer
    self.hidden_size = hidden_size
    if freeze_bert:
      for param in self.bert.parameters():
        param.requires_grad = False

    self.bigru = nn.GRU(input_size=self.bert.config.hidden_size,hidden_size=hidden_size,
                        batch_first=True, bidirectional=True)
    self.dropout = nn.Dropout(0.3)
    self.fc = nn.Linear(hidden_size * 2, num_classes)

  def forward(self, input_ids, attention_mask):
    bert_output = self.bert(input_ids=input_ids, attention_mask=attention_mask).last_hidden_state
    #https://huggingface.co/docs/transformers/model_doc/bert#transformers.BertModel - last_hidden_state = (batch_size, sequence_length, hidden_size)
    bert_output = self.dropout(bert_output)
    bigru_output, _ = self.bigru(bert_output) #https://pytorch.org/docs/stable/generated/torch.nn.GRU.html for input and output tensor details
    bigru_output = self.dropout(bigru_output)
    output = self.fc(bigru_output[:, -1, :])

    return output

In [ ]:
model = BERTBiGRUClassifier(model_bcb, tokenizer_bcb, hidden_size=config.hidden_size, num_classes=3, freeze_bert=True)
model = model.to(device)

In [ ]:
model

BERTBiGRUClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwi

### Optimizer & eval & loss functions

In [ ]:
optimizer = AdamW(model.parameters(), lr=config.lr, eps=config.epsilon)


scheduler = get_linear_schedule_with_warmup(optimizer,num_warmup_steps=0,
                                            num_training_steps=len(train_dataloader)*config.epochs)
#https://www.programcreek.com/python/example/112000/transformers.get_linear_schedule_with_warmup

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
cross_entropy  = nn.CrossEntropyLoss()

In [ ]:
def f1_score_func(preds, labels):
  preds_flat = np.argmax(preds, axis=1).flatten()
  labels_flat = labels.flatten()
  return f1_score(labels_flat, preds_flat, average='weighted')

def accuracy_per_class(preds, labels, label_dict):
  label_dict_inverse = {v: k for k, v in label_dict.items()}

  preds_flat = np.argmax(preds, axis=1).flatten()
  labels_flat = labels.flatten()

  for label in np.unique(labels_flat):
    y_preds = preds_flat[labels_flat==label]
    y_true = labels_flat[labels_flat==label]
    print(f'Class: {label_dict_inverse[label]}')
    print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

In [ ]:
def evaluate(validation_dataloader):
  model.eval()

  loss_val_total = 0
  predictions, true_vals = [], []

  for batch in validation_dataloader:
    batch = tuple(b.to(config.device) for b in batch)
    input, att_mask, labels = batch

    with torch.no_grad():
      outputs = model(input, att_mask)

      loss = cross_entropy(outputs, labels)
      loss_val_total = loss_val_total + loss.item()

      outputs = outputs.detach().cpu().numpy()
      predictions.append(outputs)
      true_vals.append(labels.cpu().numpy())

  loss_val_avg = loss_val_total/len(validation_dataloader)
  predictions = np.concatenate(predictions, axis=0)
  true_vals = np.concatenate(true_vals, axis=0)

  return loss_val_avg, predictions, true_vals

In [ ]:
def model_train(model, train_dataloader, val_dataloader):
  for epoch in tqdm(range(1, config.epochs+1)):

    model.train()
    loss_train_total = 0
    #predictions, true_vals = [], []

    #tqdm to show progress of the training
    progress_bar = tqdm(train_dataloader, desc='Epoch {:1d}'.format(epoch), leave=True)

    for batch in progress_bar:
      model.zero_grad()

      batch = tuple(b.to(config.device) for b in batch)
      input, att_mask, labels = batch

      outputs = model(input, att_mask)

      loss = cross_entropy(outputs, labels)
      loss_train_total = loss_train_total + loss.item()
      loss.backward()

      torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

      optimizer.step()
      scheduler.step()

    torch.save(model.state_dict(), f'ModelBCB_epoch_{epoch}.model')

    tqdm.write(f'\nEpoch {epoch}:')

    loss_train_avg = loss_train_total/len(train_dataloader)
    tqdm.write(f'Training loss: {loss_train_avg}')

    val_avg_loss, predictions, true_vals = evaluate(val_dataloader)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_avg_loss}')
    tqdm.write(f'F1 Score: {val_f1}')

### Model create and train

In [ ]:
train_loss = model_train(model, train_dataloader, val_dataloader)

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/13849 [00:00<?, ?it/s]


Epoch 1:
Training loss: 1.0234817373502108
Validation loss: 1.0353716719353376
F1 Score: 0.48829530260122667


Epoch 2:   0%|          | 0/13849 [00:00<?, ?it/s]


Epoch 2:
Training loss: 0.9535039012867136
Validation loss: 0.9534679869455951
F1 Score: 0.5513207350823123


...


Epoch 8:
Training loss: 0.8284896751613563
Validation loss: 0.8351548555460219
F1 Score: 0.6423015539433099


Epoch 9:   0%|          | 0/13849 [00:00<?, ?it/s]


Epoch 9:
Training loss: 0.821786156028167
Validation loss: 0.8364952825125297
F1 Score: 0.6445568667640891


Epoch 10:   0%|          | 0/13849 [00:00<?, ?it/s]


Epoch 10:
Training loss: 0.815775081479267
Validation loss: 0.8482479536353618
F1 Score: 0.6435064182396548


### Testset

In [ ]:
model.load_state_dict(torch.load('/content/ModelBCB_epoch_7.model'))#, map_location=torch.device('cpu')))


<All keys matched successfully>

In [ ]:
_, predictions, true = evaluate(val_dataloader)

In [ ]:
preds = np.argmax(predictions, axis=1).flatten()
print(classification_report(preds, true))

              precision    recall  f1-score   support

           0       0.71      0.64      0.67      6529
           1       0.61      0.53      0.57      6776
           2       0.61      0.80      0.69      4500

    accuracy                           0.64     17805
   macro avg       0.64      0.66      0.64     17805
weighted avg       0.64      0.64      0.64     17805



In [ ]:
_, predictions, true = evaluate(test_dataloader)

In [ ]:
preds = np.argmax(predictions, axis=1).flatten()
print(classification_report(preds, true))

              precision    recall  f1-score   support

           0       0.69      0.64      0.67      6403
           1       0.61      0.53      0.57      6830
           2       0.62      0.79      0.70      4572

    accuracy                           0.64     17805
   macro avg       0.64      0.66      0.64     17805
weighted avg       0.64      0.64      0.64     17805



In [ ]:
import gc
gc.collect()

98

In [ ]:
torch.cuda.empty_cache()